# Many thanks to:

"Automatic Extractive Text Summarization using TF-IDF

In the recent years, information grows rapidly along with the development of social media. With the increasing amount of information, it takes more effort and time to review the entire text document and understand its contents..."

https://medium.com/voice-tech-podcast/automatic-extractive-text-summarization-using-tfidf-3fc9a7b26f5

# **Important libraries**

In [ ]:
!pip install datasets
!pip install rouge
from rouge import Rouge
import nltk
import os
import re
import math
import operator
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
Stopwords = set(stopwords.words('english'))
wordlemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# **Load Datset**

In [ ]:
from datasets import load_dataset
test_dataset = load_dataset("scientific_papers", "arxiv", split="test[0:10]")
test_dataset

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for scientific_papers contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/scientific_papers
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Dataset({
    features: ['article', 'abstract', 'section_names'],
    num_rows: 10
})

In [ ]:
import pandas as pd
test_df = pd.DataFrame(test_dataset)
test_df.drop(columns=['section_names'], inplace=True)
test_df

,article,abstract
0,for about 20 years the problem of properties o...,the short - term periodicities of the daily s...
1,it is believed that the direct detection of gr...,we study the detectability of circular polari...
2,"as a common quantum phenomenon , the tunneling...","starting from the wkb approximation , a new b..."
3,for the hybrid monte carlo algorithm ( hmc)@xc...,we study a novel class of numerical integrato...
4,recently it was discovered that feynman integr...,new methods for obtaining functional equation...
5,one of the main goals of the search for period...,in the hierarchical search for periodic sourc...
6,this review focuses specifically on what we ha...,i summarize what we have learned about the na...
7,single - transverse spin asymmetries ( ssas ) ...,we present a phenomenological study of the si...
8,kingman s coalescent is a random tree introduc...,kingman s coalescent is a random tree that ar...
9,rapid progress in the design and manufacture o...,we discuss several novel types of multi - com...


# **Pre-processing**

In [ ]:
def remove_special_characters(text):
  # remove non-alphanumeric characters and digits
  text = re.sub(r'[^a-zA-Z\s]', ' ', str(text))
  return text


In [ ]:
def truncate(tokenized_words):
  max_length = 512
  if len(tokenized_words) > max_length:
    print(f"{len(tokenized_words)} > {max_length} Truncating...")
    text = tokenized_words[:max_length]
  return text

In [ ]:
def preprocess(text):
  text = [word for word in text if word.lower() not in Stopwords] # remove stopwords
  text = [word for word in text if len(word) > 1] # remove one letter words
  text = [word.lower() for word in text] # convert to lowercase
  text = [wordlemmatizer.lemmatize(word) for word in text] # lemmatize
  return text

# **POS Tagging**

In [ ]:
# POS-Tagging verbs and nouns.
def pos_tagging(text):
    pos_tag = nltk.pos_tag(text.split())
    pos_tagged_noun_verb = []
    for word,tag in pos_tag:
        if tag == "NN" or tag == "NNP" or tag == "NNS" or tag == "VB" or tag == "VBD" or tag == "VBG" or tag == "VBN" or tag == "VBP" or tag == "VBZ":
             pos_tagged_noun_verb.append(word)
    return pos_tagged_noun_verb

# **Count word frequency**

In [ ]:
# Counts word frequency in a document
def freq(words):
    words = [word.lower() for word in words]
    dict_freq = {}
    words_unique = []
    for word in words:
       if word not in words_unique:
           words_unique.append(word)
    for word in words_unique:
       dict_freq[word] = words.count(word)
    return dict_freq

# **TF-IDF**

In [ ]:
# TF(w) = (Number of times term w appears in a document) / (Total number of terms in the document)
def tf_score(word,sentence):
    freq_sum = 0
    word_frequency_in_sentence = 0
    len_sentence = len(sentence)
    for word_in_sentence in sentence.split():
        if word == word_in_sentence:
            word_frequency_in_sentence = word_frequency_in_sentence + 1
    tf =  word_frequency_in_sentence/ len_sentence
    return tf

# IDF(w) = log_e(Total number of documents / Number of documents with term w in it)
def idf_score(no_of_sentences,word,sentences):
    no_of_sentence_containing_word = 0
    for sentence in sentences:
        sentence = remove_special_characters(str(sentence))
        sentence = sentence.split()
        sentence = preprocess(sentence)
        if word in sentence:
            no_of_sentence_containing_word = no_of_sentence_containing_word + 1
    idf = math.log10(no_of_sentences/no_of_sentence_containing_word)
    return idf
# TFIDF = TF * IDF
def tf_idf_score(tf,idf):
    return tf*idf

# This method calls the TF_score, IDF_score, and returns the value of the TF_IDF_score.
def word_tfidf(dict_freq,word,sentences,sentence):
    word_tfidf = []
    tf = tf_score(word,sentence)
    idf = idf_score(len(sentences),word,sentences)
    tf_idf = tf_idf_score(tf,idf)
    return tf_idf

# **Calculate sentence score**

In [ ]:
# The score for each sentence decides its importance
# The score is calculated by passing a POS-Tagged sentence for verbs and nouns to the TF-IDF algorithm.
def sentence_importance(sentence,dict_freq,sentences):
     sentence_score = 0
     sentence = remove_special_characters(sentence)
     no_of_sentences = len(sentences)
     pos_tagged_sentence = []
     pos_tagged_sentence = pos_tagging(sentence)
     pos_tagged_sentence = preprocess(pos_tagged_sentence)
     for word in pos_tagged_sentence:
                sentence_score = sentence_score + word_tfidf(dict_freq,word,sentences,sentence)
     return sentence_score

# **Generate summary**

In [ ]:
def generate_summary(word_freq,tokenized_sentence):# Sorts the dictionary of sentences in a descending order with respect to the sentence score (importance).
    c = 1
    sentence_with_importance = {}
    for sent in tokenized_sentence:
      sentenceimp = sentence_importance(sent,word_freq,tokenized_sentence)
      sentence_with_importance[c] = sentenceimp
      c = c+1
    sentence_with_importance = sorted(sentence_with_importance.items(), key=operator.itemgetter(1),reverse=True)

    # Limit the generated summary to be in the range of 150 words.
    max_word_length = 150
    summary = []
    word_count = 0

    for word_prob in sentence_with_importance:
      sentence_index = word_prob[0]
      sentence = tokenized_sentence[sentence_index - 1]
      words = sentence.split()

      if word_count + len(words) <= max_word_length:
          summary.append(sentence)
          word_count += len(words)
      else:
          break

    # Print the summary.

    summary = " ".join(summary)
    print("\n")
    print("Summary:")
    print(summary)

    return summary
#print(len(summary.split()))
#outF = open('summary.txt',"w")
#outF.write(summary)

# **Model Evaluation**

In [ ]:
def get_rouge_scores3(actual_summary, predicted_summary):
    rouge = Rouge()
    scores = rouge.get_scores(predicted_summary, actual_summary)[0]  # Access the first item in the list of scores
    rouge_1_f = scores['rouge-1']['f']
    rouge_2_f = scores['rouge-2']['f']
    rouge_L_f = scores['rouge-l']['f']

    # Calculate precision, recall, and F1 scores
    precision = (scores['rouge-1']['p'] + scores['rouge-2']['p'] + scores['rouge-l']['p']) / 3
    recall = (scores['rouge-1']['r'] + scores['rouge-2']['r'] + scores['rouge-l']['r']) / 3
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0  # Avoid division by zero
    return [rouge_1_f, rouge_2_f, rouge_L_f, precision, recall, f1]


In [ ]:
# Lists to calculate peroformance scores
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
pred_summary_list = []

# Extract the 'Article' and 'Abstract' from the DataFrame
for index, row in test_df.iterrows():
   article = row["article"]
   actual_abstract = row["abstract"]
   print(f"Article {index+1}:")

   # Pre-processing and summary generation
   tokenized_sentence = sent_tokenize(article)
   article_text = remove_special_characters(article)
   tokenized_words = word_tokenize(article)
   tokenized_words = truncate(tokenized_words)
   tokenized_words = preprocess(tokenized_words)
   word_freq = freq(tokenized_words)
   abstract = generate_summary(word_freq,tokenized_sentence)

   # Calculate ROUGE scores
   rouge_scores = get_rouge_scores3(actual_abstract, abstract)
   metrics = ["ROUGE-1 F1", "ROUGE-2 F1", "ROUGE-L F1", "Precision", "Recall", "F1 Score"]
   print("\n")
   print("ROUGE Scores:")
   for i in range(len(metrics)):
      print(metrics[i] + ":", rouge_scores[i])
   print("="*40)
   print("\n")

   # Append scores to lists
   rouge1_scores.append(rouge_scores[0])
   rouge2_scores.append(rouge_scores[1])
   rougeL_scores.append(rouge_scores[2])
   precision_scores.append(rouge_scores[3])
   recall_scores.append(rouge_scores[4])
   f1_scores.append(rouge_scores[5])
   pred_summary_list.append(abstract)



Article 1:
6230 > 512 Truncating...


Summary:
let the set of values of the independent variable of the autocorrelation function be called @xmath26 and it can be divided into the sums of disjoint sets : @xmath27 where + @xmath28 + @xmath29 @xmath30 @xmath31 + @xmath32 + @xmath33 @xmath34 @xmath35 @xmath36 @xmath37 @xmath38
@xmath39 @xmath40    well , the set @xmath41 contains all integer values of @xmath23 from the interval of @xmath42 for which the autocorrelation function and the cosinus function with the period @xmath43 $ ] are positive . first of them was discovered by @xcite in the occurence rate of gamma - ray flares detected by the gamma - ray spectrometer aboard the _ solar maximum mission ( smm ) . @xcite concluded that the region of larger wavelet power shifts from 475-day ( 1.3-year ) period to 620-day ( 1.7-year ) period and then back to 475-day ( 1.3-year ) .


ROUGE Scores:
ROUGE-1 F1: 0.22459892548257038
ROUGE-2 F1: 0.062068960563852964
ROUGE-L F1: 0.18181817681946882
Pr

In [ ]:
# Add predicted summaries to test_data DataFrame
test_df["pred_summary"] = pred_summary_list

# Add scores to test_data DataFrame
scores = [rouge1_scores, rouge2_scores, rougeL_scores, precision_scores, recall_scores, f1_scores]
for i in range(len(metrics)):
  test_df[metrics[i]] = scores[i]

test_df

,article,abstract,pred_summary,ROUGE-1 F1,ROUGE-2 F1,ROUGE-L F1,Precision,Recall,F1 Score
0,for about 20 years the problem of properties o...,the short - term periodicities of the daily s...,let the set of values of the independent varia...,0.224599,0.062069,0.181818,0.156234,0.156549,0.156391
1,it is believed that the direct detection of gr...,we study the detectability of circular polari...,the following form for @xmath152 was derived i...,0.142857,0.011111,0.126984,0.093384,0.093963,0.093672
2,"as a common quantum phenomenon , the tunneling...","starting from the wkb approximation , a new b...",the @xmath0 decay half - life is related to th...,0.285714,0.058824,0.224490,0.216787,0.168609,0.189687
3,for the hybrid monte carlo algorithm ( hmc)@xc...,we study a novel class of numerical integrato...,\end{aligned}\ ] ] the second derivative of th...,0.140351,0.012195,0.105263,0.087209,0.084733,0.085953
4,recently it was discovered that feynman integr...,new methods for obtaining functional equation...,( [ 4prop_relation ] ) over a common denominat...,0.177215,0.017391,0.113924,0.105519,0.100300,0.102844
5,one of the main goals of the search for period...,in the hierarchical search for periodic sourc...,fig. fig. community . `` improved hough search...,0.180723,0.057613,0.168675,0.212304,0.099687,0.135670
6,this review focuses specifically on what we ha...,i summarize what we have learned about the na...,it is typical to subdivide cc sne into at lea...,0.250000,0.064140,0.189655,0.243839,0.128066,0.167933
7,single - transverse spin asymmetries ( ssas ) ...,we present a phenomenological study of the si...,fiz . rev . rev . lett . lett . lett . lett . ...,0.093333,0.000000,0.066667,0.045977,0.063492,0.053333
8,kingman s coalescent is a random tree introduc...,kingman s coalescent is a random tree that ar...,properties of @xmath205 : * applying the grtne...,0.231579,0.007042,0.200000,0.156313,0.137359,0.146224
9,rapid progress in the design and manufacture o...,we discuss several novel types of multi - com...,introducing the effect of diffraction in a sla...,0.166667,0.021164,0.136364,0.092428,0.130101,0.108076


In [ ]:
# Find the Average of the scores
averages = []
for metric in metrics:
    average = test_df[metric].mean()
    averages.append(average)

# Print average scores
for i in range(len(metrics)):
    print(f"Average {metrics[i]}: {averages[i]}")

Average ROUGE-1 F1: 0.18930382165564172
Average ROUGE-2 F1: 0.03115493741079501
Average ROUGE-L F1: 0.15138394389902204
Average Precision: 0.1409994773982921
Average Recall: 0.11628596300560232
Average F1 Score: 0.12397838789926185


In [ ]:
text ="It has been shown that approximately 90% of subjects who die through suicide are diagnosed with a psychiatric disorder prior to their death [1]. Suicide is particularly worrisome in Bipolar Disorder (BD), given the high prevalence of this disorder and the strong association between suicide and depressive symptoms [2]. In addition, a large cohort study found that among men, the absolute risk of suicide was higher in BD (7.8%) compared to any other psychiatric condition. Among women, BD was associated with the second highest risk, at 4.8%, just below schizophrenia at 4.9% [3]. Furthermore, patients with BD showed twice the rate of suicide risk than patients with Major Depressive Disorder and about 20 to 30 times greater risk than in the general population [4].It is known that at least 90% of suicide cases occur in people with a psychiatric disorder, with mood disorders being the most prevalent condition in this scenario [1]. In addition, many of the individuals are untreated or inadequately treated [5]. Suicide prediction tools may aid health professionals to identify subjects at-risk for suicidal behavior, thus enabling not only early intervention and personalized care, but also the development of new strategies for suicide prevention. As there are already interventions available that show anti-suicidal effects, such as lithium for mood disorders [6] and clozapine for schizophrenia [7], adequate detection and treatment of these conditions may help reduce suicide rates.Virginia Woolf was a British novelist who biographers suggest suffered from BD [8]. From 1910 to 1913, Woolf was ill on several occasions [9]. During her life, she made at least three suicide attempts. She received the habitual treatments of the time, such as “rest cure therapy at home,” which consisted of gaining weight, sleeping, and “the resting of the intellect,” which also meant a recommendation not to write [10]. Virginia endured several depressive and manic episodes until she took her life [11] on March 28, 1941 during a depressive episode [10].Virginia Woolf presented several known risk factors associated with suicide in patients with BD [12] such as early traumatic experiences (sexual abuse), psychotic symptoms, family history of suicide, and a higher number of depressive episodes. Nevertheless, a meta-analysis of 365 studies showed that traditional suicide risk factors are poor predictors of a future suicide attempt [13]. A possible explanation for this finding is that complex patterns of interaction between risk factors are not considered in traditional statistics [14,15]. Moreover, the risk factors identified so far come from group-level results, and do not wield satisfactory predictive results at an individual level [13]. Thus, it is paramount to develop better models to predict suicidal behavior that include these interactions and explore different levels of information. Machine learning, a field of artificial intelligence that focuses on algorithms that can learn from data and then extract patterns to make new assumptions from unseen information, are increasingly being used in behavioral sciences to provide predictive models for clinical practice [16]. Machine learning can handle an enormous amount of data, such as text data, and combine them in nonlinear and highly interactive ways [17]. One important question is whether suicidal behavior is associated with an identifiable writing pattern. Virginia Woolf left a vast written repertoire contained in her diaries, where she wrote freely about her feelings, providing a living record of her past mood states.Machine-learning studies already published include the use of natural processing language to detect emotions present in suicide notes [18] and classify them [19], to detect suicidality in Twitter activity [20], and to extract a particular emotion in a suicide note sentence [21,22]. Similarly, it appears that electronic health records may also be of value to predict suicidal behavior [13,22]. Besides text classification, previous studies used machine-learning algorithms to predict suicide. For instance, a study reported a clinical signature by using a relevance vector machine to identify suicidality in patients with mood disorders [2]. The most relevant predictor variables in distinguishing attempters from non-attempters were previous hospitalizations for depression, lifetime history of psychosis, cocaine dependence, and post-traumatic stress disorder comorbidity [2]. Another study that used machine-learning algorithms showed that individual unrest, personal satisfaction, and reasons for living are the variables most associated with suicidality [23]. They concluded that these variables could be used to create an actionable assessment tool that may identify individual risk and protective factors [23]The present study aims to analyze whether text classification coupled with machine-learning algorithms can predict unfavorable outcomes, such as suicide, using the written records of a single individual. In order to test this hypothesis, we used the content of Virginia Woolf’s diaries and letters to assess whether there is a text signature in her writings prior to her suicide.MethodsThis is a text classification study with a machine-learning approach. We used a Naïve Bayes algorithm. It is worth mentioning that Naïve Bayes is a Bayesian method that estimates the probability of an event’s occurrence [17]. Although Naïve Bayes is not the only machine-learning method that utilizes Bayesian statistics, it is one of the most common. This is particularly true for text classification, where it has become the de facto standard [17]. For instance, this algorithm is commonly used to classify e-mails in spam or ham messages [17]. This algorithm was selected because 1) it requires relatively few examples for training, but also works well with very large numbers of examples; 2) it provides the estimated probability for a prediction; and 3) it is very effective and performs well with noisy data. Therefore, by using Naïve Bayes, we compared Virginia Woolf’s texts written over the 60 days before her death versus texts written outside this period to identify a signature associated with suicide. We included letters and diaries written by Virginia Woolf. We excluded books, novels, short stories, and article fragments."
